In [1]:
!pip install timm scikit-learn numpy pandas tqdm torch torchvision

In [2]:
import os
from google.colab import drive

# Bu kısım sadece Colab'da çalışır
try:
    print("Google Drive'a bağlanılıyor...")
    # Colab'da kimlik doğrulama penceresi açılır
    drive.mount('/content/drive')
    print("Drive bağlantısı başarılı.")

    # Veri setinizin Google Drive'daki ana yolu (EN, AR, TR alt klasörlerini içerir)
    # Lütfen Drive'daki yolu kontrol edin ve gerekirse güncelleyin.
    DATA_DIR = '/content/drive/MyDrive/data_v1'

except Exception as e:
    # Hata durumunda veya Colab dışında çalışıyorsa
    print(f"Drive bağlantı hatası veya Colab ortamı algılanmadı: {e}")
    DATA_DIR = './data_v1' # Yerel denemeler için yedek yol

# Veri klasörünün varlığını kontrol et
if not os.path.exists(DATA_DIR):
    # Bu, kullanıcıya Drive bağlantısının başarılı olsa bile veri yolunun yanlış olduğunu bildirir
    raise FileNotFoundError(f"Hata: '{DATA_DIR}' yolu bulunamadı. Lütfen Google Drive'daki veri klasörünün doğru konumda olduğundan emin olun.")
else:
    print(f"Veri klasörü başarıyla bulundu: {DATA_DIR}")

Google Drive'a bağlanılıyor...
Mounted at /content/drive
Drive bağlantısı başarılı.
Veri klasörü başarıyla bulundu: /content/drive/MyDrive/data_v1


In [3]:
import os
import random
import json
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score
import numpy as np
import time
from datetime import datetime

# --- YARDIMCI FONKSİYON: SÜRE FORMATLAMA ---

def format_duration(seconds):
    """Saniye cinsinden süreyi (H saat, M dakika, S saniye) formatına dönüştürür."""
    seconds = int(round(seconds))
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return f"{h:02d} saat, {m:02d} dakika, {s:02d} saniye"

# --- YAPILANDIRMA VE YOL TANIMLARI ---

# LÜTFEN KONTROL EDİN: Drive üzerindeki verilerinizin ana klasör yolu
ROOT_DIR = '/content/drive/MyDrive/data_v1'
# EĞİTİLMİŞ MODELLERİN KAYDEDİLECEĞİ KLASÖR YOLU (Swin yerine ConvNeXt olarak güncellendi)
DRIVE_MODEL_DIR = '/content/drive/MyDrive/convnext_image_models_colab'

# ConvNeXt Model Mimarisi (Swin yerine ConvNeXt Tiny seçildi)
CONVNEXT_MODEL_NAME = 'convnext_tiny' # İstenirse 'convnext_base', 'convnext_large' vb. ile değiştirilebilir.

# Dil ve Sınıf Yapılandırmaları: Bu harita, hangi dil klasöründe hangi alt klasörlerin (sınıfların) aranacağını belirler.
language_info = {
    # TR: Kullanıcının verdiği TSL sınıf listesi
    'TR': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z'],
    # AR: Kullanıcının önceki dosyasındaki Arap alfabesi işaretleri
    'AR': ['ain', 'al', 'aleff', 'bb', 'dal', 'dha', 'dhad', 'fa', 'gaaf', 'ghain', 'ha', 'haa', 'jeem', 'kaaf', 'khaa', 'la', 'laam', 'meem', 'nun', 'ra', 'saad', 'seen', 'sheen', 'taa', 'taad', 'tha', 'tah', 'ya', 'yaa', 'zay', 'zal'],
    # EN: Standart 26 ASL harfi
    'EN': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'],
}
ALL_LANG_CODES = ['EN', 'AR', 'TR'] # Tüm diller için eğitim yapılacak

# Eğitim parametreleri
NUM_EPOCHS = 50         # Colab GPU için varsayılan olarak korundu
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
TEST_SIZE = 0.2
RANDOM_STATE = 42

# --- MODEL TANIMLAMA (ConvNeXt için güncellendi) ---

def create_model(num_classes, pretrained=True):
    """ConvNeXt modelini Timm kullanarak tanımlar ve ağırlıkları yükler."""
    print(f"Timm Kütüphanesinden {CONVNEXT_MODEL_NAME} Yükleniyor...")

    # Pretrained=True ile ImageNet ağırlıklarını yükle
    # ConvNeXt, modern bir CNN mimarisidir ve timm'de desteklenir.
    model = timm.create_model(CONVNEXT_MODEL_NAME, pretrained=pretrained, num_classes=num_classes)

    # Çıkış katmanı (classifier) otomatik olarak num_classes'e ayarlanmıştır.
    return model


class SignLanguageDataset(Dataset):
    """İşaret dili görüntüleri için özel veri kümesi sınıfı."""
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            # Görüntüyü aç ve RGB formatına çevir (Hata durumunda PIL'den kaynaklanabilir)
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            # Hatalı/bozuk resimleri raporla ve atla (döngüyü bozmamak için)
            print(f"Uyarı: Resim yüklenemedi {img_path}: {e}")
            # Bu durumun DataLoader'da hata vermesine neden olabileceğini unutmayın,
            # ancak bu kodu basitleştirilmiş haliyle koruyoruz.
            raise e

        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# --- VERİ HAZIRLAMA FONKSİYONU ---

def prepare_lang_data(root_dir, lang_code, test_size=TEST_SIZE, random_state=RANDOM_STATE):
    """Sadece belirtilen dilin resimlerini toplar ve etiketler."""
    all_image_paths = []
    all_labels = []

    lang_path = os.path.join(root_dir, lang_code)
    class_list = language_info.get(lang_code, [])

    if not os.path.exists(lang_path) or not class_list:
        print(f"Uyarı: '{lang_code}' diline ait klasör ({lang_path}) veya sınıf bilgisi bulunamadı. Bu dil atlanıyor.")
        return None, None, None, None, None

    # language_info'da tanımlanan her bir sınıf (alt klasör) için resimleri topla
    for char_dir in class_list:
        char_path = os.path.join(lang_path, char_dir)
        if os.path.isdir(char_path):
            label = char_dir
            for img_name in os.listdir(char_path):
                img_path = os.path.join(char_path, img_name)
                # Resim dosyalarını kontrol et (küçük/büyük harf duyarsız)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    all_image_paths.append(img_path)
                    all_labels.append(label)
        else:
            print(f"Uyarı: '{lang_code}' - '{char_dir}' sınıf klasörü bulunamadı.")

    print(f"\nVeri Kontrolü: '{lang_code}' için Toplam {len(all_labels)} resim bulundu.")
    if len(all_labels) == 0:
        print(f"Hata: '{lang_code}' dizininde hiç resim bulunamadı. Atlanıyor.")
        return None, None, None, None, None

    # Sınıfları sayısal etiketlere dönüştür
    le = LabelEncoder()
    encoded_labels = le.fit_transform(all_labels)

    # Veri setini eğitim ve test olarak bölme (stratify ile sınıf dağılımını korur)
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_image_paths, encoded_labels, test_size=test_size, random_state=random_state, stratify=encoded_labels
    )

    # Sınıf haritasını oluştur ve kaydet
    class_map = {name: int(label) for label, name in enumerate(le.classes_)} # {index: class_name}
    # Dosya yolu ConvNeXt için güncellendi
    class_map_path = os.path.join(DRIVE_MODEL_DIR, f'class_map_{lang_code}.json')
    os.makedirs(os.path.dirname(class_map_path), exist_ok=True)
    try:
        with open(class_map_path, 'w', encoding='utf-8') as f:
            json.dump(class_map, f, ensure_ascii=False, indent=4)
            print(f"\n{lang_code} için {len(class_map)} sınıf haritası '{class_map_path}' dosyasına kaydedildi.")
    except Exception as e:
        print(f"Uyarı: Sınıf haritası kaydedilemedi: {e}")


    return train_paths, test_paths, train_labels, test_labels, le.classes_

# Veri zenginleştirme (Data Augmentation) ve normalizasyon adımları (ConvNeXt için standart)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    # ConvNeXt/ImageNet için standart normalizasyon değerleri
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Doğrulama/Test için kullanılan transform (Sadece yeniden boyutlandırma ve normalizasyon)
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# --- MODEL DEĞERLENDİRME ---

def evaluate_model(model, test_loader, device, class_names):
    """Verilen modelin test verisi üzerindeki performansını değerlendirir."""
    model.eval()
    all_preds = []
    all_labels = []

    # Test/Değerlendirme döngüsünde tqdm'i kullanmak görsel geri bildirim için iyidir.
    test_bar = tqdm(test_loader, desc="[Test]", leave=False)
    with torch.no_grad():
        for images, labels in test_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    # Raporu oluştururken sınıf isimlerini kullan
    report = classification_report(all_labels, all_preds, target_names=class_names, zero_division=0, digits=4)

    return accuracy, report

# --- TEK DİL EĞİTİM FONKSİYONU ---

def train_single_language_model(lang_code, train_paths, test_paths, train_labels, test_labels, class_names):
    """Tek bir dil için modeli eğitir, kaydeder ve raporlar."""

    print(f"\n{'='*20} {lang_code} DİLİ İÇİN EĞİTİM BAŞLIYOR ({len(class_names)} Sınıf) {'='*20}")

    # Log dosyasının yolu (ConvNeXt için güncellendi)
    epoch_log_path = os.path.join(DRIVE_MODEL_DIR, f'convnext_{lang_code}_epoch_details.txt')

    # DataLoader'ları oluştur
    train_dataset = SignLanguageDataset(train_paths, train_labels, transform)
    test_dataset = SignLanguageDataset(test_paths, test_labels, val_transform)

    # Colab'da GPU kullanırken num_workers'ı artırmak veri yükleme hızını artırır
    num_workers_to_use = 4 if torch.cuda.is_available() else 0
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=num_workers_to_use)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=num_workers_to_use)

    num_classes = len(class_names)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ConvNeXt Modeli Oluştur (fonksiyon adı güncellendi)
    model = create_model(num_classes).to(device)

    # Optimizasyon ve Kayıp Fonksiyonu
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE) # ConvNeXt için de AdamW yaygındır
    criterion = nn.CrossEntropyLoss()

    # Log dosyası için başlık yazma (ConvNeXt için güncellendi)
    with open(epoch_log_path, 'w', encoding='utf-8') as f:
        f.write(f"--- CONVNEXT EĞİTİM DETAYLI EPOCH LOGLARI ({lang_code}) ---\n")
        f.write(f"Model: {CONVNEXT_MODEL_NAME}, Epochs: {NUM_EPOCHS}, LR: {LEARNING_RATE}\n")
        f.write(f"Kullanılan Cihaz: {device}\n")
        f.write(f"Başlangıç Tarih/Saat: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"Veri Seti Boyutu (Eğitim): {len(train_dataset)}, (Test): {len(test_dataset)}\n")
        f.write(f"Toplam Sınıf Sayısı: {num_classes}\n")
        f.write("-" * 70 + "\n\n")


    # --- EĞİTİM DÖNGÜSÜ ---
    for epoch in range(NUM_EPOCHS):
        epoch_start_time = time.time()
        epoch_start_datetime = datetime.now().strftime("%H:%M:%S")

        model.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [{lang_code} Eğitimi]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            train_bar.set_postfix({'Kayıp': f'{running_loss / (train_bar.n + 1):.4f}'})

        avg_train_loss = running_loss / len(train_loader)

        # --- EPOCH SONU DEĞERLENDİRME ---

        # evaluate_model, Test doğruluğunu ve sınıflandırma raporunu döndürür
        overall_accuracy, overall_report = evaluate_model(model, test_loader, device, class_names)

        epoch_end_time = time.time()
        epoch_end_datetime = datetime.now().strftime("%H:%M:%S")
        epoch_duration = epoch_end_time - epoch_start_time

        # Süreyi HH saat, MM dakika, SS saniye formatına dönüştür
        formatted_epoch_duration = format_duration(epoch_duration)

        # Ekrana Özet Rapor Yazdır
        print(f"\n|--- EPOCH {epoch+1}/{NUM_EPOCHS} Raporu ({lang_code}) ---")
        print(f"| Süre: {formatted_epoch_duration} | Eğt. Kaybı: {avg_train_loss:.4f} | Test Doğruluğu: {overall_accuracy:.4f}")
        print("-" * 70)


        # --- MODEL VE LOG KAYDETME ---

        # 1. Modeli kaydet (pth) - ConvNeXt için güncellendi
        model_filename = f'convnext_{lang_code}_epoch_{epoch+1}.pth'
        model_path = os.path.join(DRIVE_MODEL_DIR, model_filename)
        torch.save(model.state_dict(), model_path)

        # 2. Detaylı Epoch Raporunu kaydet (txt)
        with open(epoch_log_path, 'a', encoding='utf-8') as f:
            f.write(f"| --- EPOCH {epoch+1}/{NUM_EPOCHS} Raporu ({lang_code}) ---\n")
            f.write(f"| Başlangıç Saati: {epoch_start_datetime}\n")
            f.write(f"| Bitiş Saati: {epoch_end_datetime}\n")
            f.write(f"| Süre: {formatted_epoch_duration}\n") # İSTENEN FORMAT
            f.write(f"| Ortalama Eğitim Kaybı: {avg_train_loss:.4f}\n")
            f.write(f"| Test Doğruluğu: {overall_accuracy:.4f}\n")
            f.write("| Sınıflandırma Raporu:\n")

            # Classification Report çıktısını satır satır log dosyasına yaz
            for line in overall_report.split('\n'):
                f.write(f"| {line}\n")

            f.write("| ------------------------------------------------------------------\n\n")

        print(f"Model, '{model_path}' olarak kaydedildi.")
        print(f"Detaylı rapor '{epoch_log_path}' dosyasına eklendi.")
        print("-" * 70)


    # --- DİL BAZLI BİTİŞ RAPORU ---
    print(f"\n{lang_code} DİLİ EĞİTİMİ BAŞARIYLA TAMAMLANDI.")
    print(f"Son Model: {model_path}")


# --- ANA EĞİTİM BORU HATTI ---

def run_training_pipeline():
    """Tüm tanımlı diller için eğitim sürecini yönetir."""

    global_start_time = time.time()
    os.makedirs(DRIVE_MODEL_DIR, exist_ok=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if device.type == 'cuda':
        print(f"Kullanılan Cihaz: {torch.cuda.get_device_name(0)}")
    else:
        print(f"UYARI: Sadece CPU/RAM mevcut. Eğitim çok uzun sürecektir. Colab'da GPU'yu etkinleştirin. Kullanılan Cihaz: {device}")

    all_language_stats = {}

    # Tüm diller için eğitim başlatılıyor
    for lang_code in ALL_LANG_CODES:
        lang_start_time = time.time()

        # Veri Hazırlama
        train_paths, test_paths, train_labels, test_labels, class_names = prepare_lang_data(ROOT_DIR, lang_code)

        if train_paths is None:
            continue

        # Modeli Eğitme
        train_single_language_model(lang_code, train_paths, test_paths, train_labels, test_labels, class_names)

        lang_duration = time.time() - lang_start_time
        all_language_stats[lang_code] = format_duration(lang_duration)

        print(f"\n--- {lang_code} İşlemleri Tamamlandı. Toplam Süre: {all_language_stats[lang_code]} ---\n")


    # --- GLOBAL BİTİŞ RAPORU ---
    global_duration = time.time() - global_start_time
    formatted_global_duration = format_duration(global_duration)

    print("\n\n" + "="*70)
    print("TÜM DİL MODELLERİNİN EĞİTİMİ BAŞARIYLA TAMAMLANDI.")
    print("="*70)
    for lang, duration in all_language_stats.items():
        print(f"- {lang} Dili Toplam Eğitim Süresi: {duration}")
    print("-" * 70)
    print(f"Tüm Projenin Toplam Süresi: {formatted_global_duration}")

# Eğitimi başlat
if __name__ == '__main__':
    run_training_pipeline()

Kullanılan Cihaz: NVIDIA A100-SXM4-40GB

Veri Kontrolü: 'EN' için Toplam 39000 resim bulundu.

EN için 26 sınıf haritası '/content/drive/MyDrive/convnext_image_models_colab/class_map_EN.json' dosyasına kaydedildi.

==================== EN DİLİ İÇİN EĞİTİM BAŞLIYOR (26 Sınıf) ====================
Timm Kütüphanesinden convnext_tiny Yükleniyor...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

KeyboardInterrupt: 